To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

!pip install pymupdf

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from pathlib import Path
import fitz  # PyMuPDF
from datasets import Dataset
import re

# --- Parameters ---
finetuning_dir = Path("/content/drive/MyDrive/finetuning")
max_lines = 10000
instruction = """You are a low-level PDF code generator.
Given raw PDF page content streams (e.g., BT/ET text drawing blocks), your job is to rewrite each stream for accessibility.
Only return the improved PDF stream. Do not explain or comment.
Do not interpret the data. Do not describe the content. Do not wrap in markdown or text blocks.
Each input stream will be followed by your output stream."""

# --- Helper to extract /Contents stream from each page ---
def extract_content_streams(pdf_path):
    doc = fitz.open(pdf_path)
    contents = []
    for page in doc:
        contents_obj = page.get_contents()
        if isinstance(contents_obj, list):
            for xref in contents_obj:
                stream = doc.xref_stream(xref)
                if stream:
                    contents.append(stream.decode(errors="ignore"))
        elif contents_obj:
            stream = doc.xref_stream(contents_obj)
            if stream:
                contents.append(stream.decode(errors="ignore"))
    doc.close()
    return "\n".join(contents)

# --- Create instruction-style dataset ---
examples = []

for bf_path in finetuning_dir.glob("*_bf_uncompressed.pdf"):
    base = bf_path.name.replace("_bf_uncompressed.pdf", "")
    src_path = finetuning_dir / f"{base}_uncompressed.pdf"

    try:
        if not src_path.exists():
            print(f"Warning: Missing source for {bf_path.name}, skipping.")
            continue

        src_code = extract_content_streams(src_path)
        tgt_code = extract_content_streams(bf_path)

        # Truncate to max lines
        src_lines = "\n".join(src_code.splitlines()[:max_lines])
        tgt_lines = "\n".join(tgt_code.splitlines()[:max_lines])

        # Prepare escaped previews
        input_preview = src_lines[:200].replace("\n", "\\n")
        output_preview = tgt_lines[:200].replace("\n", "\\n")

        # Print the inputs and outputs being added
        print(
            f"Adding example {base}:\n"
            "--- Input (first 200 chars) ---\n"
            f"{input_preview}\n"
            "--- Output (first 200 chars) ---\n"
            f"{output_preview}"
        )

        examples.append({
            "instruction": instruction,
            "input": src_lines,
            "output": tgt_lines
        })

    except FileNotFoundError:
        print(f"Error: File not found: {src_path}. Skipping.")
    except Exception as e:
        print(f"Error processing pair {bf_path.name}: {e}")

print(f"Prepared {len(examples)} examples.")

# --- Huggingface Dataset + Uns­loth Prompt Formatting ---
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    return {
        "text": [
            alpaca_prompt.format(i, inp, out) + EOS_TOKEN
            for i, inp, out in zip(examples["instruction"], examples["input"], examples["output"])
        ]
    }

dataset = Dataset.from_list(examples)
dataset = dataset.map(formatting_prompts_func, batched=True)

Adding example 2018-08_Poster-Ersteinschaetzung_UT-1:
--- Input (first 200 chars) ---
\nq\n0 0 1190.551 1683.78 re\nW\nn\n/GS0 gs\n/CS0 cs\n.898 0 .312 scn\n1 0 0 1 70.984299 918.553589 cm\n0 0 m\n0 -41.858 l\n0 -48.486 5.373 -53.858 12 -53.858 c\n449.02 -53.858 l\n455.647 -53.858 461.02 -48.486 46
--- Output (first 200 chars) ---
\n/Artifact /PR0 BDC\nq\n0 0 1190.551 1683.78 re\nW\nn\n/GS0 gs\n/CS0 cs\n0.898 0 0.312 scn\n1 0 0 1 70.984299 918.553589 cm\n0 0 m\n0 -41.858 l\n0 -48.486 5.373 -53.858 12 -53.858 c\n449.02 -53.858 l\n455.647 -53.
Adding example SSgS_Ich-mag-mich_Kern:
--- Input (first 200 chars) ---
\nq\n0 0 325.984009 325.984009 re\nW\nn\n/GS0 gs\n328.161199 0 0 328.161199 -1.097435 -1.119876 cm\nq\n/Im0 Do\nQ\nQ\nq\n0 0 325.984009 325.984009 re\nW\nn\n/GS1 gs\n/Fo0 Do\nQ\nq\n47.764 244.505005 232.380997 44.487 re
--- Output (first 200 chars) ---
\n/Artifact /PR0 BDC\nq\n0 0 325.984009 325.984009 re\nW\nn\n/GS0 gs\n328.161199 0 0 328.161199 -1.097435 -1.119876 cm\nq\n/Im0 

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [12]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/97 [00:00<?, ? examples/s]

In [13]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
7.822 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 97 | Num Epochs = 5 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.648100
2,1.630200
3,1.610800
4,1.479400
5,1.633600
6,1.661000
7,1.553800
8,1.474900
9,1.552700
10,1.543700


In [15]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

268.3244 seconds used for training.
4.47 minutes used for training.
Peak reserved memory = 7.822 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 19.774 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [18]:
import random

testing_dir = Path("/content/drive/MyDrive/testing")

# List non-barrier-free PDFs (exclude _bf_uncompressed)
src_pdfs = list(finetuning_dir.glob("*_uncompressed.pdf")) # set aside testing directory in future
src_pdfs = [p for p in src_pdfs if not str(p.name).endswith("_bf_uncompressed.pdf")]

# Randomly pick one
random_pdf = random.choice(src_pdfs)
print(f"Selected PDF: {random_pdf.name}")

Selected PDF: AUVA-AA-6-23_uncompressed.pdf


In [24]:
src_code_testing = extract_content_streams(random_pdf)
input_text = "\n".join(src_code.splitlines()[:max_lines])

instruction = """You are a low-level PDF code generator.
Given raw PDF page content streams (e.g., BT/ET text drawing blocks), your job is to rewrite each stream for accessibility.
Only return the improved PDF stream. Do not explain or comment.
Do not interpret the data. Do not describe the content. Do not wrap in markdown or text blocks.
Each input stream will be followed by your output stream."""

prompt = alpaca_prompt.format(instruction, input_text, "")

inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(response)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a low-level PDF code generator.
Given raw PDF page content streams (e.g., BT/ET text drawing blocks), your job is to rewrite each stream for accessibility.
Only return the improved PDF stream. Do not explain or comment.
Do not interpret the data. Do not describe the content. Do not wrap in markdown or text blocks.
Each input stream will be followed by your output stream.

### Input:

q
0 0 595.32 841.92 re
W*
n
1.9368 -.000001 0.89653 64.26 732.56 cm
/Fo0 Do
Q
q
60.984 724.06 127.34 81.624 re
W*
n
BT
/F0 12 Tf
203.69 732.58 Td
<0003>Tj
ET
Q
BT
/F1 12 Tf
1.1875 0 0 1 254.69 779.52 Tm
[<00250044>-9<0056>-7<004c0056>-7<0047>-3<0052>-11<004e0058>-11<0050>-3<0048>-7<0051>-3<0057>-3<0048>-19.6<0003>]TJ
-23.242019 -13.799997 Td
[<003d>-3<0048>-7<00550057004c0049>-5<004c005d>-13<004c0048>-7<00550058>-10<0051>-3<0

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [25]:
src_code_testing = extract_content_streams(random_pdf)
input_text = "\n".join(src_code.splitlines()[:max_lines])
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction,
        input_text,
        ""  # leave output blank
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a low-level PDF code generator.
Given raw PDF page content streams (e.g., BT/ET text drawing blocks), your job is to rewrite each stream for accessibility.
Only return the improved PDF stream. Do not explain or comment.
Do not interpret the data. Do not describe the content. Do not wrap in markdown or text blocks.
Each input stream will be followed by your output stream.

### Input:

q
0 0 595.32 841.92 re
W*
n
1.9368 -.000001 0.89653 64.26 732.56 cm
/Fo0 Do
Q
q
60.984 724.06 127.34 81.624 re
W*
n
BT
/F0 12 Tf
203.69 732.58 Td
<0003>Tj
ET
Q
BT
/F1 12 Tf
1.1875 0 0 1 254.69 779.52 Tm
[<00250044>-9<0056>-7<004c0056>-7<0047>-3<0052>-11<004e0058>-11<0050>-3<0048>-7<0051>-3<0057>-3<0048>-19.6<0003>]TJ
-23.242019 -13.799997 Td
[<003d>-3<0048>-7<00550057004c0049>-5<004c005d>-13<004c0048>-7<00550

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [26]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if True: model.push_to_hub_merged("heseltiNe/model0", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("heseltiNe/model0", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("heseltiNe/model0", token = "")
    tokenizer.push_to_hub("heseltiNe/model0", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [36]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model0", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if True: model.push_to_hub_gguf("heseltiNe/model0gguf", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 48.56 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 63.73it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at heseltiNe/model0gguf into q8_0 GGUF format.
The output location will be /content/heseltiNe/model0gguf/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model0gguf
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to

Uploading...:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/heseltiNe/model0gguf


Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
